<a href="https://colab.research.google.com/github/MOOwuttichai/BSC_DPDM2023/blob/main/%E0%B8%A7%E0%B8%B4%E0%B9%80%E0%B8%84%E0%B8%A3%E0%B8%B2%E0%B8%B0%E0%B8%AB%E0%B9%8C%E0%B8%82%E0%B9%89%E0%B8%AD%E0%B8%A1%E0%B8%B9%E0%B8%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [335]:
!pip install pythainlp
!pip install -U sentence-transformers

In [336]:
from pythainlp.corpus import thai_stopwords
from pythainlp.tokenize import Tokenizer
from pythainlp.util import normalize
from pythainlp.corpus.common import thai_words
from sklearn.cluster import KMeans
import pandas as pd
import ast

In [337]:
kok = pd.read_csv('/content/drive/MyDrive/เชี่ยงใหม่ (1)/Facebook_pre.csv',encoding='utf-8')

In [338]:
kok

,name,comments,ยอดไลค์,จำนวนการตอบกลับ,จำนวนคำ,token,defind_cancer_with_nlp,use_ful,defind_exp_with_python,defind_Genden_with_nlp,...,อุจจาระเป็นเลือด,อ่อนเพลีย,เจ็บปวด,เบื่ออาหาร,เสียงแหบ,เหนื่อยง่าย,แผลเรื้อรัง,ไข้เรื้อรัง,ไม่มีการระบุอาการ,ไอเรื้อรัง
0,99แคมป์,ไม่ทานเนื้อค่ะ เห็นญาติพี่น้องรอบข้างเป็นมะ เ...,0,0,13,"['ทาน', 'เนื้อ', 'ญาติพี่น้อง', 'รอบข้าง', 'มะ...",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,ไม่สามารถระบุได้,ไม่ระบุเพศ,...,0,0,0,0,0,0,0,0,1,0
1,A Achi Sarakam,ปี62 แม่เกร็ดเลือดต่ำ+ค่าตับผิดปกติ ตรวจพบไวรั...,0,0,95,"['ปี', '62', 'แม่', 'เกร็ด', 'เลือด', 'ต่ำ', '...",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,เล่าประสบการณ์คนอื่น,เพศหญิง,...,0,0,0,0,0,0,0,0,1,0
2,A-Chiraya Kasiyapong,Bureerat Saisudta,0,0,3,"['Bureerat', 'Saisudta', 'end']",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,ไม่สามารถระบุได้,เพศหญิง,...,0,0,0,0,0,0,0,0,1,0
3,AI'Jack TheRam,ดีมากๆเลยครับ จะได้เป็นวิทยาทานให้คนอื่น พ่อกั...,0,0,88,"['ดีมาก', 'วิทยาทาน', 'คนอื่น', 'พ่อ', 'แม่', ...",มะเร็งเต้านม,non,เล่าประสบการณ์คนอื่น,เล่าทั้งสองเพศ,...,0,0,0,0,0,0,0,0,1,0
4,AS Kate-Duan,.,0,0,1,['end'],ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,ไม่สามารถระบุได้,เล่าทั้งสองเพศ,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2119,ไอ มิส ยู,อันนี้ไม่รู้จริงๆนะคะ เราไม่สามารถตรวจครั้งแรก...,0,1,51,"['อันนี้', 'รู้', 'นะคะ', 'ตรวจ', 'ครั้งแรก', ...",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,ไม่สามารถระบุได้,เล่าทั้งสองเพศ,...,0,0,0,0,0,0,0,0,1,0
2120,ไอด้า ไอรดาา,สมปอง มานุ,0,0,5,"['สม', 'ปอง', 'นุ', 'end']",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,ไม่สามารถระบุได้,เล่าทั้งสองเพศ,...,0,0,0,0,0,0,0,0,1,0
2121,ไอลดา ปรีเอี่ยม,ชาวสวน ส้ม มาอ่าน,0,0,6,"['ชาวสวน', 'ส้ม', 'อ่าน', 'end']",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,ไม่สามารถระบุได้,เล่าทั้งสองเพศ,...,0,0,0,0,0,0,0,0,1,0
2122,杨小龙,คุณแม่พึ่งพบว่าเป็นมะเร็งรังไข่ตอนตุลาคมปีนี้ค...,0,1,83,"['คุณแม่', 'มะเร็ง', 'รังไข่', 'ตอน', 'ตุลาคม'...",มะเร็งรังไข่,non,เล่าประสบการณ์คนอื่น,เล่าทั้งสองเพศ,...,0,0,0,0,0,0,0,0,0,0


In [339]:
len(kok[kok['defind_cancer_with_nlp']!='ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น'])

573

In [340]:
a=kok[kok['defind_cancer_with_nlp']!='ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น']

In [341]:
data = pd.read_csv('/content/drive/MyDrive/เชี่ยงใหม่ (1)/REDDIT_pre.csv',encoding='utf-8')

In [342]:
comment=data

In [343]:
# comment=data.groupby('name').sum().reset_index()
# # สร้าง set ข้อมูลภาษาไทย
# words = set(thai_words())
# words.remove("โรคมะเร็ง")
# name =['กระเพาะปัสสวะ','กระเพาะปัสสาวะ','เยื่อบุโพรงมดลูก','ปากมดลูก','เม็ดเลือดขาว','กระเพาะอาหาร','กระเพราะอาหาร','ต่อมไทรอยด์','ต่อมไทยรอยด์','ท่อน้ำดี']
# for i in name:
#     words.add(i)
# def remove_stopthai(tokens):
#     final = [word.lower()
#             for word in tokens if word not in thai_stopwords()]
#     return final
# def remove_bark(tokens):
#     final = [word.lower() for word in tokens if word not in [' ','(',')','@','#','-','...','.','=','+','..','1','2','3','4','5','6','7','8','9','0','']]
#     return final
# # สร้าง list เก็บตัว nlp เพิ่อนำไปวิเคราะห์โรค อาการ เเละเพศ
# list_token =[]
# count = []
# for i in range(len(comment)):#len(comment)
#     text= comment['comments'][i]
#     custom_tokenizer = Tokenizer(words)
#     Token = custom_tokenizer.word_tokenize(normalize(str(text)))
#     Token.append('end')
#     Token_final = remove_bark(remove_stopthai(Token))
#     list_token.append(Token_final)
#     count.append(len(Token_final))
# comment['จำนวนคำ'] = count
# comment['token'] = list_token

In [348]:
import string
import nltk
nltk.download(['punkt', 'wordnet', 'omw-1.4', 'stopwords'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
from textblob import TextBlob
def token(data):
    tokens = nltk.word_tokenize(data)
    tokens.insert(0, 'start')
    tokens.append('end')
    return tokens
def remove_(tokens):
    final = [word.lower()
            for word in tokens if word not in stopwords.words("english")]
    return final
def remove_bark(tokens):
    final = [word.lower() for word in tokens if word not in [' ','(',')','@','#','-','...','.','=','+','..','1','2','3','4','5','6','7','8','9','0','',"’"]]
    return final
# Lemmatizing
def lemma(final):
    # initialize an empty string
    str1 = ' '.join(final)
    s = TextBlob(str1)
    lemmatized_sentence = " ".join([w.lemmatize() for w in s.words])
    return final
data_com = comment['comments'].to_list()
list_token_red = []
for item_comment_red in range(len(data_com)):
    data_r_token = token(data_com[item_comment_red].translate(str.maketrans('', '', string.punctuation)))
    data_r_token = remove_(data_r_token)
    data_r_token = remove_bark(data_r_token)
    data_r_token = lemma(data_r_token)
    list_token_red.append(data_r_token)
    count_1=[]
    for item in list_token_red:
        count_1.append(len(item))
comment['จำนวนคำ'] = count_1
comment['token'] = list_token_red

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [349]:
comment

,name,comments,จำนวนคำ,ยอดไลค์,จำนวนการตอบกลับ,token,defind_cancer_with_nlp,defind_Genden_with_nlp,defind_Genden_with_python,defind_exp_with_python,...,chronic fever,fatigue,feverishness,headache,itching,numbness,pain,seizures,skin ulcers,vomiting blood
0,-Suriel-,Congratulations and good on you for persisting...,222,0,0,"[start, congratulations, good, persisting, til...",Unable to identify / not sure if it is,เพศหญิง,ไม่สามารถระบุได้,เล่าประสบการณ์ตัวเอง,...,0,0,0,0,0,0,0,0,0,0
1,0livesworld,How many rounds of chemo did you do? I had 5 r...,30,0,0,"[start, how, many, rounds, chemo, i, rounds, c...",Unable to identify / not sure if it is,เพศหญิง,เพศชาย,เล่าประสบการณ์คนอื่น,...,0,0,0,0,0,0,0,0,0,0
2,17voltaire89,Congratulations and here’s to continuing the s...,36,0,0,"[start, congratulations, continuing, survival,...",Unable to identify / not sure if it is,เพศหญิง,เพศชาย,เล่าประสบการณ์คนอื่น,...,0,0,0,0,0,0,0,0,0,0
3,19keightyfour,Was diagnosed at 22 with Hodgkin Lymphoma and ...,33,0,0,"[start, was, diagnosed, 22, hodgkin, lymphoma,...",Unable to identify / not sure if it is,เพศหญิง,เพศชาย,เล่าประสบการณ์คนอื่น,...,0,0,0,0,0,0,0,0,0,0
4,1LungWonder,18 years post-treatment... every day I have so...,215,0,0,"[start, 18, years, posttreatment, every, day, ...",Unable to identify / not sure if it is,เล่าทั้งสองเพศ,ไม่สามารถระบุได้,เล่าประสบการณ์ตัวเอง,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,zeerorequiem,"Never had APL, but I did have ALL. I totally r...",140,0,0,"[start, never, apl, i, all, i, totally, relate...",Unable to identify / not sure if it is,เล่าทั้งสองเพศ,ไม่สามารถระบุได้,เล่าประสบการณ์ตัวเอง,...,0,0,0,0,0,0,0,0,0,0
1131,zeleno1,I'm really really sorry. I forever resent my i...,43,0,0,"[start, im, really, really, sorry, i, forever,...",Unable to identify / not sure if it is,เล่าทั้งสองเพศ,ไม่สามารถระบุได้,เล่าประสบการณ์ตัวเอง,...,0,0,0,0,0,0,0,0,0,0
1132,zeppelinsbabe,I’m terrified of this day but wow,6,0,0,"[start, i, terrified, day, wow, end]",Unable to identify / not sure if it is,เล่าทั้งสองเพศ,ไม่ระบุเพศ,ไม่สามารถระบุได้,...,0,0,0,0,0,0,0,0,0,0
1133,zeshef,This is the exact same condition that led my p...,43,0,0,"[start, this, exact, condition, led, partner, ...",Unable to identify / not sure if it is,เล่าทั้งสองเพศ,ไม่สามารถระบุได้,เล่าประสบการณ์ตัวเอง,...,0,0,0,0,0,0,0,0,0,0


In [374]:
set_cancer=list(set(comment['defind_cancer_with_nlp']))

In [375]:
set_cancer

['bone cancer',
 'skin cancer',
 'liver cancer',
 'Tell more than 2 diseases.',
 'ovarian cancer',
 'pancreatic cancer',
 'brain cancer',
 'breast cancer',
 'stomach cancer',
 'Unable to identify / not sure if it is',
 'kidney cancer',
 'cervical cancer',
 'prostate cancer',
 'bladder cancer',
 'endometrial cancer',
 'thyroid cancer',
 'penile cancer',
 'lung cancer',
 'esophageal cancer',
 'uterine cancer']

#วิธีการรักษา

In [376]:
comment

,name,comments,จำนวนคำ,ยอดไลค์,จำนวนการตอบกลับ,token,defind_cancer_with_nlp,defind_Genden_with_nlp,defind_Genden_with_python,defind_exp_with_python,...,fatigue,feverishness,headache,itching,numbness,pain,seizures,skin ulcers,vomiting blood,Treatment
0,-Suriel-,Congratulations and good on you for persisting...,222,0,0,"[start, congratulations, good, persisting, til...",Unable to identify / not sure if it is,เพศหญิง,ไม่สามารถระบุได้,เล่าประสบการณ์ตัวเอง,...,0,0,0,0,0,0,0,0,0,[ไม่สามารถระบุได้]
1,0livesworld,How many rounds of chemo did you do? I had 5 r...,30,0,0,"[start, how, many, rounds, chemo, i, rounds, c...",Unable to identify / not sure if it is,เพศหญิง,เพศชาย,เล่าประสบการณ์คนอื่น,...,0,0,0,0,0,0,0,0,0,[Chemotherapy]
2,17voltaire89,Congratulations and here’s to continuing the s...,36,0,0,"[start, congratulations, continuing, survival,...",Unable to identify / not sure if it is,เพศหญิง,เพศชาย,เล่าประสบการณ์คนอื่น,...,0,0,0,0,0,0,0,0,0,[ไม่สามารถระบุได้]
3,19keightyfour,Was diagnosed at 22 with Hodgkin Lymphoma and ...,33,0,0,"[start, was, diagnosed, 22, hodgkin, lymphoma,...",Unable to identify / not sure if it is,เพศหญิง,เพศชาย,เล่าประสบการณ์คนอื่น,...,0,0,0,0,0,0,0,0,0,"[Chemotherapy, Radiation Therapy]"
4,1LungWonder,18 years post-treatment... every day I have so...,215,0,0,"[start, 18, years, posttreatment, every, day, ...",Unable to identify / not sure if it is,เล่าทั้งสองเพศ,ไม่สามารถระบุได้,เล่าประสบการณ์ตัวเอง,...,0,0,0,0,0,1,0,0,0,"[Surgery, Radiation Therapy]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,zeerorequiem,"Never had APL, but I did have ALL. I totally r...",140,0,0,"[start, never, apl, i, all, i, totally, relate...",Unable to identify / not sure if it is,เล่าทั้งสองเพศ,ไม่สามารถระบุได้,เล่าประสบการณ์ตัวเอง,...,0,0,0,0,0,0,0,0,0,[ไม่สามารถระบุได้]
1131,zeleno1,I'm really really sorry. I forever resent my i...,43,0,0,"[start, im, really, really, sorry, i, forever,...",Unable to identify / not sure if it is,เล่าทั้งสองเพศ,ไม่สามารถระบุได้,เล่าประสบการณ์ตัวเอง,...,0,0,0,0,0,0,0,0,0,[ไม่สามารถระบุได้]
1132,zeppelinsbabe,I’m terrified of this day but wow,6,0,0,"[start, i, terrified, day, wow, end]",Unable to identify / not sure if it is,เล่าทั้งสองเพศ,ไม่ระบุเพศ,ไม่สามารถระบุได้,...,0,0,0,0,0,0,0,0,0,[ไม่สามารถระบุได้]
1133,zeshef,This is the exact same condition that led my p...,43,0,0,"[start, this, exact, condition, led, partner, ...",Unable to identify / not sure if it is,เล่าทั้งสองเพศ,ไม่สามารถระบุได้,เล่าประสบการณ์ตัวเอง,...,0,0,0,0,0,0,0,0,0,[ไม่สามารถระบุได้]


In [377]:
comment['token'][1]

['start',
 'how',
 'many',
 'rounds',
 'chemo',
 'i',
 'rounds',
 'chemotherapy',
 'hodgkins',
 'lymphoma',
 '18',
 'personally',
 'dont',
 'struggle',
 'chemo',
 'brain',
 'side',
 'effects',
 'i',
 'know',
 'lot',
 'survivors',
 'chemo',
 'confusing',
 'different',
 'depending',
 'type',
 'cancer',
 'treated',
 'end']

In [378]:
fix_word = []
for i in range(len(comment)):
  try:
    x = ast.literal_eval(comment['token'][i])
    fix_word.append(x)
    comment['token']=fix_word
  except:
    pass

In [379]:
comment['token'][1]

['start',
 'how',
 'many',
 'rounds',
 'chemo',
 'i',
 'rounds',
 'chemotherapy',
 'hodgkins',
 'lymphoma',
 '18',
 'personally',
 'dont',
 'struggle',
 'chemo',
 'brain',
 'side',
 'effects',
 'i',
 'know',
 'lot',
 'survivors',
 'chemo',
 'confusing',
 'different',
 'depending',
 'type',
 'cancer',
 'treated',
 'end']

In [380]:
x =[]
for i in range(len(comment)):
  if 'อายุ' in comment['token'][i]:
    x.append(comment['token'][i])
print(x)

[]


In [381]:
Treatment ={'Surgery':['ผ่าตัด','Surgery','ผ่า','operation', 'incision', 'surgery'],'Chemotherapy':['chemotherapy','chemo','คีโม','เคมี','herceptin'],'Radiation Therapy':['radiation','ฉายแสง','เเร่','ฉายเเสง',r'radi.*'],'Targeted Therapy':['targeted','เป้า']}

In [382]:
Treatment_names = ['Surgery','Chemotherapy','Radiation Therapy','Targeted Therapy']

In [383]:
Treatment['Surgery']

['ผ่าตัด', 'Surgery', 'ผ่า', 'operation', 'incision', 'surgery']

In [384]:
k = []
for i in range(len(comment)):
  L=[]
  for j in Treatment_names:
    for a in Treatment[j]:
      if a in comment['token'][i]:
        L.append(j)
      else:
        L.append('ไม่สามารถระบุได้')
  L =list(set(L))
  k.append(L)
comment['Treatment'] = k


In [385]:
# Treatment = ['ผ่าตัด','คีโม','ฉายแสง','แร่']
k = []
for i in range(len(comment)):
  L=[]
  for j in Treatment_names:
    for lol in Treatment[j]:
      if lol in comment['token'][i]:
        L.append(j)
      else:
        L.append('ไม่สามารถระบุได้')
  L =list(set(L))
  if len(L) >= 2:
    L.remove('ไม่สามารถระบุได้')
  k.append(L)
comment['Treatment'] = k

In [386]:
len(k)

1135

In [387]:
comment[comment['Treatment'] == 'ผ่าตัด']

,name,comments,จำนวนคำ,ยอดไลค์,จำนวนการตอบกลับ,token,defind_cancer_with_nlp,defind_Genden_with_nlp,defind_Genden_with_python,defind_exp_with_python,...,fatigue,feverishness,headache,itching,numbness,pain,seizures,skin ulcers,vomiting blood,Treatment


In [388]:
df= pd.Series(comment['Treatment'])

In [389]:
len(comment)

1135

In [390]:
Treatment_dumm = comment['Treatment'].str.join(sep='*').str.get_dummies(sep='*')
comment_data = comment.join(Treatment_dumm)

In [391]:
comment_data

,name,comments,จำนวนคำ,ยอดไลค์,จำนวนการตอบกลับ,token,defind_cancer_with_nlp,defind_Genden_with_nlp,defind_Genden_with_python,defind_exp_with_python,...,pain,seizures,skin ulcers,vomiting blood,Treatment,Chemotherapy,Radiation Therapy,Surgery,Targeted Therapy,ไม่สามารถระบุได้
0,-Suriel-,Congratulations and good on you for persisting...,222,0,0,"[start, congratulations, good, persisting, til...",Unable to identify / not sure if it is,เพศหญิง,ไม่สามารถระบุได้,เล่าประสบการณ์ตัวเอง,...,0,0,0,0,[ไม่สามารถระบุได้],0,0,0,0,1
1,0livesworld,How many rounds of chemo did you do? I had 5 r...,30,0,0,"[start, how, many, rounds, chemo, i, rounds, c...",Unable to identify / not sure if it is,เพศหญิง,เพศชาย,เล่าประสบการณ์คนอื่น,...,0,0,0,0,[Chemotherapy],1,0,0,0,0
2,17voltaire89,Congratulations and here’s to continuing the s...,36,0,0,"[start, congratulations, continuing, survival,...",Unable to identify / not sure if it is,เพศหญิง,เพศชาย,เล่าประสบการณ์คนอื่น,...,0,0,0,0,[ไม่สามารถระบุได้],0,0,0,0,1
3,19keightyfour,Was diagnosed at 22 with Hodgkin Lymphoma and ...,33,0,0,"[start, was, diagnosed, 22, hodgkin, lymphoma,...",Unable to identify / not sure if it is,เพศหญิง,เพศชาย,เล่าประสบการณ์คนอื่น,...,0,0,0,0,"[Chemotherapy, Radiation Therapy]",1,1,0,0,0
4,1LungWonder,18 years post-treatment... every day I have so...,215,0,0,"[start, 18, years, posttreatment, every, day, ...",Unable to identify / not sure if it is,เล่าทั้งสองเพศ,ไม่สามารถระบุได้,เล่าประสบการณ์ตัวเอง,...,1,0,0,0,"[Surgery, Radiation Therapy]",0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,zeerorequiem,"Never had APL, but I did have ALL. I totally r...",140,0,0,"[start, never, apl, i, all, i, totally, relate...",Unable to identify / not sure if it is,เล่าทั้งสองเพศ,ไม่สามารถระบุได้,เล่าประสบการณ์ตัวเอง,...,0,0,0,0,[ไม่สามารถระบุได้],0,0,0,0,1
1131,zeleno1,I'm really really sorry. I forever resent my i...,43,0,0,"[start, im, really, really, sorry, i, forever,...",Unable to identify / not sure if it is,เล่าทั้งสองเพศ,ไม่สามารถระบุได้,เล่าประสบการณ์ตัวเอง,...,0,0,0,0,[ไม่สามารถระบุได้],0,0,0,0,1
1132,zeppelinsbabe,I’m terrified of this day but wow,6,0,0,"[start, i, terrified, day, wow, end]",Unable to identify / not sure if it is,เล่าทั้งสองเพศ,ไม่ระบุเพศ,ไม่สามารถระบุได้,...,0,0,0,0,[ไม่สามารถระบุได้],0,0,0,0,1
1133,zeshef,This is the exact same condition that led my p...,43,0,0,"[start, this, exact, condition, led, partner, ...",Unable to identify / not sure if it is,เล่าทั้งสองเพศ,ไม่สามารถระบุได้,เล่าประสบการณ์ตัวเอง,...,0,0,0,0,[ไม่สามารถระบุได้],0,0,0,0,1


In [392]:
# prompt: ต้องการหาคำ ที่ขึ้นต้นด้วย Radi เเล้วด้านหลังเอาอะไรก็ได้

import re
radi_words = []
for i in range(len(comment)):
  for word in comment['token'][i]:
    if re.match(r'Radi.*', word):
      radi_words.append(word)
print(radi_words)


[]


In [393]:
comment_data

,name,comments,จำนวนคำ,ยอดไลค์,จำนวนการตอบกลับ,token,defind_cancer_with_nlp,defind_Genden_with_nlp,defind_Genden_with_python,defind_exp_with_python,...,pain,seizures,skin ulcers,vomiting blood,Treatment,Chemotherapy,Radiation Therapy,Surgery,Targeted Therapy,ไม่สามารถระบุได้
0,-Suriel-,Congratulations and good on you for persisting...,222,0,0,"[start, congratulations, good, persisting, til...",Unable to identify / not sure if it is,เพศหญิง,ไม่สามารถระบุได้,เล่าประสบการณ์ตัวเอง,...,0,0,0,0,[ไม่สามารถระบุได้],0,0,0,0,1
1,0livesworld,How many rounds of chemo did you do? I had 5 r...,30,0,0,"[start, how, many, rounds, chemo, i, rounds, c...",Unable to identify / not sure if it is,เพศหญิง,เพศชาย,เล่าประสบการณ์คนอื่น,...,0,0,0,0,[Chemotherapy],1,0,0,0,0
2,17voltaire89,Congratulations and here’s to continuing the s...,36,0,0,"[start, congratulations, continuing, survival,...",Unable to identify / not sure if it is,เพศหญิง,เพศชาย,เล่าประสบการณ์คนอื่น,...,0,0,0,0,[ไม่สามารถระบุได้],0,0,0,0,1
3,19keightyfour,Was diagnosed at 22 with Hodgkin Lymphoma and ...,33,0,0,"[start, was, diagnosed, 22, hodgkin, lymphoma,...",Unable to identify / not sure if it is,เพศหญิง,เพศชาย,เล่าประสบการณ์คนอื่น,...,0,0,0,0,"[Chemotherapy, Radiation Therapy]",1,1,0,0,0
4,1LungWonder,18 years post-treatment... every day I have so...,215,0,0,"[start, 18, years, posttreatment, every, day, ...",Unable to identify / not sure if it is,เล่าทั้งสองเพศ,ไม่สามารถระบุได้,เล่าประสบการณ์ตัวเอง,...,1,0,0,0,"[Surgery, Radiation Therapy]",0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,zeerorequiem,"Never had APL, but I did have ALL. I totally r...",140,0,0,"[start, never, apl, i, all, i, totally, relate...",Unable to identify / not sure if it is,เล่าทั้งสองเพศ,ไม่สามารถระบุได้,เล่าประสบการณ์ตัวเอง,...,0,0,0,0,[ไม่สามารถระบุได้],0,0,0,0,1
1131,zeleno1,I'm really really sorry. I forever resent my i...,43,0,0,"[start, im, really, really, sorry, i, forever,...",Unable to identify / not sure if it is,เล่าทั้งสองเพศ,ไม่สามารถระบุได้,เล่าประสบการณ์ตัวเอง,...,0,0,0,0,[ไม่สามารถระบุได้],0,0,0,0,1
1132,zeppelinsbabe,I’m terrified of this day but wow,6,0,0,"[start, i, terrified, day, wow, end]",Unable to identify / not sure if it is,เล่าทั้งสองเพศ,ไม่ระบุเพศ,ไม่สามารถระบุได้,...,0,0,0,0,[ไม่สามารถระบุได้],0,0,0,0,1
1133,zeshef,This is the exact same condition that led my p...,43,0,0,"[start, this, exact, condition, led, partner, ...",Unable to identify / not sure if it is,เล่าทั้งสองเพศ,ไม่สามารถระบุได้,เล่าประสบการณ์ตัวเอง,...,0,0,0,0,[ไม่สามารถระบุได้],0,0,0,0,1


12 = มะเร็งกล่องเสียง
19 = มะเร็งไทรอยด์
31 = มะเร็งโพรงจมูก
มะเร็งไขกระดูก


In [432]:
# prompt: กำหนดให้ x=[1,2,'3',4,5] ต้องการลบค่า 1 ,'3',4

x = [1, 2, '3', 4, 5,5,5,5,5]
x.remove(5)
print(x) # Output: [2, 5]


[1, 2, '3', 4, 5, 5, 5, 5]


In [427]:
x=[1,2,'3',4,5]
x.remove('3',4,5)
x

TypeError: list.remove() takes exactly one argument (3 given)

#เเบ่งคำ

In [439]:
# prompt: สร้าง list ที่ x = [1,1,2,3,4]  จากนั้น ให้ลบ 1 ออกจาก list นี้ทั้งหมด

x = [1,1,2,3,4]
while 1 in x:
  x.remove(1)
print(x)


[2, 3, 4]


In [436]:
def remove_items(test_list, item):
    # using list comprehension to perform the task
    res = [i for i in test_list if i != item]
    return res

In [442]:
from nltk.probability import FreqDist
import plotly.graph_objects as go
import plotly.express as px
lung_cancer=comment['token']#[comment['defind_cancer_with_nlp']=='มะเร็งปอด'].reset_index()
sum_word = []
for i in range(len(lung_cancer)):
    sum_word+=lung_cancer[i]
while 'end' in sum_word:
  sum_word.remove('end')
while 'i' in sum_word:
  sum_word.remove('i')
try:
  while 'start' in sum_word:
    sum_word.remove('start')
except:
  pass
fdist = FreqDist(sum_word)
x=fdist.most_common(10)
word_count_table = pd.DataFrame(x,columns=['word','count'])

In [444]:
fig = px.bar(word_count_table, x='count', y='word',text_auto=True,orientation='h')
fig.show()

#จัดกลุ่ม (sentence_transformers)

In [396]:
from sentence_transformers import SentenceTransformer,util
import numpy as np
sentences = list(comment['comments'])
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
embeddings = model.encode(sentences)
#Normalize the embeddings to unit length
Normalize_embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
clustering_model = KMeans(n_clusters=4)
clustering_model.fit(embeddings)
cluster_assignment = clustering_model.labels_
print(cluster_assignment)
print(len(cluster_assignment))

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



[2 1 1 ... 0 1 0]
1135


In [397]:
clusternd_sentences= {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
  if cluster_id not in clusternd_sentences:
    clusternd_sentences[cluster_id] = []

  clusternd_sentences[cluster_id].append(sentences[sentence_id])

In [398]:
clusternd_sentences[1]

["How many rounds of chemo did you do? I had 5 rounds of chemotherapy for Hodgkins Lymphoma at 18 and personally don't struggle with chemo brain or any side effects but I know a lot of survivors do. Chemo is confusing and can be so different depending on the type of cancer being treated.",
 'Congratulations and here’s to continuing the survival journey. Stage 4 renal cancer with metastases into lungs and lymph nodes here. 3.5 years but it’s becoming a struggle now. Just started oxygen at night. Stories such as yours help me in the dark times that we all visit occasionally. Like. Right. Now. I wish you so much continued success and happiness.',
 'Was diagnosed at 22 with Hodgkin Lymphoma and went through 16 rounds of chemo and 30 days of radiation. I’ll be 40 in December and can tell you the feelings you have never go away. It’s kind of like the waxing and waning of the moon; it’s always there, but the degree to which we experience the light changes.',
 '18 years post-treatment... every

#รวม Reddit

In [399]:
# # prompt: ทำการ concat Data_scraper_save (6) ถึง  Data_scraper_save (17)

# import pandas as pd

# # Assuming your files are named Data_scraper_save (6).csv, Data_scraper_save (7).csv, ... , Data_scraper_save (17).csv
# # Replace with your actual file paths if needed

# df_list = []
# for i in range(6, 114):
#   file_path = f'/content/Data_scraper_save ({i}).csv'  # Replace with your actual file path
#   try:
#     df = pd.read_csv(file_path, encoding='utf-8')
#     df_list.append(df)
#   except FileNotFoundError:
#     print(f"File not found: {file_path}")

# # Concatenate all DataFrames
# final_df = pd.concat(df_list, ignore_index=True)
# final_df_drop = final_df.drop_duplicates(subset=['comments'])
# # Now you have a single DataFrame containing data from all files
# print(final_df_drop)


In [400]:
# final_df_drop.to_csv('reddit_twothosan.csv',index=False)

In [401]:
# final_df_drop.groupby('name').sum().reset_index()

In [402]:
# # prompt: จากข้อมูล final_df_drop ช่วยสร้าง list เก็บชื่อที่ซ้ำ

# duplicate_names = []
# for name, group in final_df_drop.groupby('name'):
#   if len(group) > 1:
#     duplicate_names.append(name)


In [403]:
# final_df_drop[final_df_drop['name']=='myersmjsc']

# เพิ่มเติม/note

In [404]:
for i in range(len(comment)):
  print('ผ่าตัด' in comment['token'][i])

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [405]:
k = pd.DataFrame()
l = []
for i in range(len(comment)):
  if 'แนะนำ' in comment['token'][i]:
    l.append(comment['token'][i])
k['word'] = l
k

,word


In [406]:
df1 = pd.read_csv('/content/Data_scraper_save (5).csv', encoding='utf-8')
df2 = pd.read_csv('/content/Data_Docter_ศ.นพ.ชวลิต.csv', encoding='utf-8')
df3 = pd.read_csv('/content/Data_โรคร้ายผ่านได้ถ้าไม่ยอมแพ้.csv', encoding='utf-8')

FileNotFoundError: [Errno 2] No such file or directory: '/content/Data_scraper_save (5).csv'

In [ ]:
# prompt: รวม df1 df2 df3

# Concatenate the DataFrames
combined_df = pd.concat([df1, df2, df3], ignore_index=True)
combined_df.to_csv('data_Facebook.csv', index=False, encoding='utf-8')

In [ ]:
combined_df

In [ ]:
# prompt: ช่วยเเปลง '[1,2,3,4]' เป็น list [1,2,3,4]

import ast
list_string = [1,2,3,4]
try:
  list_actual = ast.literal_eval(list_string)
except:
  list_actual = list_string
print(list_actual)
